In [5]:
import os
import requests
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
import yfinance as yf

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
)

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for. Example query: Stock Market Symbol for Apple Company")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find to the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)
    
class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company. Example: AAPL, TSLA",
    )

class CompanyOverviewTool(BaseTool):
    name="CompanyOverviewTool"
    description="""
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        ticker = yf.Ticker(symbol)
        return ticker.info

class CompanyIncomeStatementTool(BaseTool):
    name="CompanyIncomeStatementTool"
    description="""
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        ticker = yf.Ticker(symbol)
        return ticker.income_stmt

class CompanyStockPerformanceTool(BaseTool):
    name="CompanyStockPerformance"
    description="""
    Use this to get the 1 month performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        ticker = yf.Ticker(symbol)
        return ticker.history(period="1mo")

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Tesla stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Tesla'}`


The new Model 3 Performance leverages the latest Noise, Vibration, and Harshness (NVH) and comfort improvements of the new Model 3, plus bespoke chassis hardware that deliver performance and stiffness improvement while reducing mass, including adaptive dampers with hollow damper piston rods, multi-sectioned stabilizer bar, ultra-high strength ... The renewed concerns about the Autopilot system came on the heals of Tesla's quarterly net profit tumbling 55% to $1.3 billion as revenue fell 9% from the first quarter of 2023. Tesla also trimmed ... Tesla and Uber have also been working together to share data in order to better deploy charging stations in New York City, where there are a lot of Uber drivers using Tesla vehicles. Add Electrek ... Tesla has been promising a $25,000 electric car since 2018, but the project has faced many challenges and setbacks. Find out the latest on th

{'input': "Give me financial information on Tesla stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "### Tesla, Inc. (TSLA) Financial Overview\n\n#### Company Overview\n- **Industry**: Auto Manufacturers\n- **Sector**: Consumer Cyclical\n- **Headquarters**: Austin, Texas, USA\n- **Website**: [Tesla](https://www.tesla.com)\n- **Market Cap**: $700.48 billion\n- **Employees**: 140,473\n\n#### Key Financial Metrics\n- **Current Price**: $219.27\n- **52-Week Range**: $138.80 - $278.98\n- **Trailing P/E Ratio**: 61.77\n- **Forward P/E Ratio**: 70.73\n- **Price to Sales Ratio**: 7.35\n- **Price to Book Ratio**: 10.54\n- **Profit Margin**: 12.996%\n- **Return on Equity**: 20.861%\n- **Debt to Equity Ratio**: 18.606\n- **Total Revenue (Latest Year)**: $96.77 billion\n- **Net Income (Latest Year)**: $14.99 billion\n- **Earnings Per Share (EPS)**: $4.31 (Diluted)\n\n#### Income Statement Highlights (Latest 